<a href="https://colab.research.google.com/github/Shufen-Yin/Artificial-Intelligence/blob/main/Assignment14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Task 1 Data preparation
#  1.1 Load dataset
# import libraries
!pip install fairlearn

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from fairlearn.metrics import MetricFrame, selection_rate, false_positive_rate, true_positive_rate
import matplotlib.pyplot as plt


In [ ]:
df = pd.read_csv('adult.csv')

print(df.head())

print("Dataset shape:", df.shape)

In [ ]:
# 1.2 Basic cleaning: handle missing values

print(df.isnull().sum())

# In the Adult dataset, missing values are as "?"
df = df.replace("?", np.nan)
df = df.dropna()

print("Dataset shape after dropping missing rows:", df.shape)
df.head()


In [ ]:
df.info()

In [ ]:
# 2 Preprocess the dataset by selecting features (X) and the target variable (y).
# 2.1 Define target variable (y)
# Convert income (<=50K, >50K) to binary 0/1
df["income_binary"] = df["income"].apply(lambda x: 1 if ">50K" in str(x) else 0)
y = df["income_binary"]

print("Target value counts:")
print(y.value_counts())



In [ ]:
# 2.2 Define feature matrix (X)
feature_cols = [
    "age",
    "educational-num",
    "hours-per-week",
    "capital-gain",
    "capital-loss",
    "workclass",
    "marital-status",
    "occupation",
    "relationship",
    "race",
    "gender"
]

X = df[feature_cols]

print("Feature columns:")
print(X.columns)
X.head()


In [ ]:
# 3. Define sensitive attribute for fairness analysis
# Here we use 'gender' as the sensitive attribute
sensitive_attr = "gender"
A = df[sensitive_attr]

print("Sensitive attribute:", sensitive_attr)
print(A.value_counts())


In [ ]:
# Task 2 Model Training and Evaluation:
# 1.1 Split the dataset into training and testing sets.
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.3,
    random_state=42,
    stratify=y
)

print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)


In [ ]:
# 1.2 Preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

numeric_features = [
    "age",
    "educational-num",
    "hours-per-week",
    "capital-gain",
    "capital-loss"]

categorical_features = [
    "workclass",
    "marital-status",
    "occupation",
    "relationship",
    "race",
    "gender"   ]

numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown="ignore")

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features)])


In [ ]:
# 2.1Train a logistic regression model using scikit-learn.

# Logistic Regression Model
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

log_reg_clf = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", LogisticRegression(max_iter=1000))
])

log_reg_clf.fit(X_train, y_train)

print("Model training completed.")


In [ ]:
# 3 Evaluation：Accuracy + Confusion Matrix + Classification Report
# Model Evaluation
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

y_pred = log_reg_clf.predict(X_test)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy)

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

# Classification Report
print("Classification Report:")
print(classification_report(y_test, y_pred))


In [ ]:
# Task 3 Fairness analysis
# 1.1 Fairness - Prepare sensitive feature on test set
# We re-split gender along with X and y to keep them aligned
from sklearn.model_selection import train_test_split

sensitive_feature = df["gender"]  # this is the sensitive attribute

X_train, X_test, y_train, y_test, A_train, A_test = train_test_split(
    X,
    y,
    sensitive_feature,
    test_size=0.3,
    random_state=42,
    stratify=y
)

print("A_test value counts:")
print(A_test.value_counts())


In [ ]:
# Re-fit the model after re-splitting
log_reg_clf.fit(X_train, y_train)


In [ ]:
# Install Fairlearn
!pip install fairlearn


In [ ]:
# Fairness - Imports
from fairlearn.metrics import (
    MetricFrame,
    selection_rate,
    false_positive_rate,
    true_positive_rate
)


In [ ]:
# 2 Fairness - Compute group metrics with MetricFrame
from sklearn.metrics import accuracy_score

# Predicted labels on test set
y_pred = log_reg_clf.predict(X_test)

# Overall accuracy (should be close to 0.8547)
overall_accuracy = accuracy_score(y_test, y_pred)
print("Overall Test Accuracy:", overall_accuracy)

# Define metrics dictionary for MetricFrame
metrics = {
    "selection_rate": selection_rate,
    "false_positive_rate": false_positive_rate,
    "true_positive_rate": true_positive_rate,
    "accuracy": accuracy_score
}

# Create MetricFrame grouped by gender (A_test)
mf = MetricFrame(
    metrics=metrics,
    y_true=y_test,
    y_pred=y_pred,
    sensitive_features=A_test
)

print("Overall metrics (aggregated):")
print(mf.overall)

print("\nMetrics by gender group:")
print(mf.by_group)


In [ ]:
# Fairness - Custom bar plots with Matplotlib
import matplotlib.pyplot as plt

# Convert by_group to DataFrame
group_metrics = mf.by_group

print(group_metrics)

# Plot selection rate, FPR, TPR in one figure each
for metric_name in ["selection_rate", "false_positive_rate", "true_positive_rate"]:
    group_metrics[metric_name].plot(kind="bar")
    plt.title(f"{metric_name} by gender")
    plt.xlabel("Gender")
    plt.ylabel(metric_name)
    plt.xticks(rotation=0)
    plt.show()


In [ ]:
ax = group_metrics["accuracy"].plot(kind="bar")
ax.set_title("accuracy by gender")
ax.set_xlabel("Gender")
ax.set_ylabel("accuracy")


In [ ]:
# Task 4 Explainability Analysis:
# 1.1 Apply SHAP (SHapley Additive exPlanations) for global and local explainability
#  Explainability - Install SHAP and LIME
!pip install shap lime
# ===== Explainability - Imports =====
import shap

In [ ]:
#1.2 SHAP - Prepare background data and feature names

# Use training data as background; sample a subset for efficiency
X_train_sample = X_train.sample(n=200, random_state=42)  # you can adjust n

# Get feature names after preprocessing (for plots)
# We need to fit the preprocessor separately to extract names
preprocessor = log_reg_clf.named_steps["preprocessor"]

# Fit preprocessor on full training data (if not already fitted)
preprocessor.fit(X_train)

# Get transformed feature names (numeric + one-hot categorical)
numeric_features = preprocessor.transformers_[0][2]
categorical_features = preprocessor.transformers_[1][2]

# OneHotEncoder is the second transformer
ohe = preprocessor.named_transformers_["cat"]
ohe_feature_names = ohe.get_feature_names_out(categorical_features)

all_feature_names = np.concatenate([numeric_features, ohe_feature_names])
print("Number of features after preprocessing:", len(all_feature_names))


In [ ]:
# 1.3 SHAP - Build KernelExplainer for the pipeline

# Define a wrapper to get probability of positive class from the pipeline

# Safe model wrapper for DataFrame input
feature_cols = X_train.columns  # original feature names

def model_predict_proba(X_input):
    """
    SHAP may pass a numpy array. We need to convert it back to a DataFrame
    with the same column names as the original training data.
    """
    import numpy as np
    import pandas as pd

    if isinstance(X_input, np.ndarray):
        X_df = pd.DataFrame(X_input, columns=feature_cols)
    else:
        # already a DataFrame
        X_df = X_input

    return log_reg_clf.predict_proba(X_df)[:, 1]

# Create a KernelExplainer using a small background set
X_train_sample = X_train.sample(n=50, random_state=42)  # keep as DataFrame

explainer = shap.KernelExplainer(
    model_predict_proba,
    X_train_sample
)

In [ ]:
#  SHAP - Compute shap_values for a test subset
X_test_sample = X_test.sample(n=300, random_state=0)  # DataFrame

shap_values = explainer.shap_values(X_test_sample, nsamples=100)

# summary plot
shap.summary_plot(
    shap_values,
    X_test_sample,
    feature_names=X_test_sample.columns
)


In [ ]:
# 2.1 Use LIME (Local Interpretable Model-agnostic Explanations) for detailed interpretation of individual predictions.
#  LIME - Imports
from lime.lime_tabular import LimeTabularExplainer

In [ ]:
# 2.2Identify numeric and categorical columns
numeric_cols = ['age', 'educational-num', 'hours-per-week']  # adjust
categorical_cols = [col for col in X_train.columns if col not in numeric_cols]

# One-hot encode categorical columns
ohe = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')
X_train_cat = ohe.fit_transform(X_train[categorical_cols])
X_test_cat = ohe.transform(X_test[categorical_cols])

# Concatenate numeric columns
X_train_num = X_train[numeric_cols].values
X_test_num = X_test[numeric_cols].values
X_train_lime = np.hstack([X_train_num, X_train_cat])
X_test_lime = np.hstack([X_test_num, X_test_cat])

# Feature names
feature_names = numeric_cols + list(ohe.get_feature_names_out(categorical_cols))


In [ ]:
# 2.3. Build LIME Tabular Explainer
lime_explainer = LimeTabularExplainer(
    training_data=X_train_lime,
    feature_names=feature_names,
    class_names=["<=50K", ">50K"],
    discretize_continuous=True,
    mode="classification",
    random_state=42
)

In [ ]:
# 2.4 Explain a single test instance
# Train a Logistic Regression on manually one-hot encoded data

log_reg_manual = LogisticRegression(max_iter=5000)
log_reg_manual.fit(X_train_lime, y_train)  # use manually one-hot encoded X_train

# 2.5 Explain a single test instance with LIME
i = 0  # choose which test instance
i = 0  # choose which test instance
lime_exp = lime_explainer.explain_instance(
    data_row=X_test_lime[i],          # use manually one-hot encoded test row
    predict_fn=log_reg_manual.predict_proba,  # call the manually trained model
    num_features=10
)

In [ ]:
# Print explanation
print("LIME explanation for test instance", i)
print(lime_exp.as_list())
